# 1.0 Data Wrangling <a name="1.1_datawrangling"></a>


## Contents <a name="contents"></a>

* [1.1 Data Wrangling](#1.1_datawrangling)

    * [Contents](#contents)
<a/>
* [1.2 Introduction](#1.2_introduction)

    * [1.2.1 Problem Recap](#1.2.1_recap)

    * [1.2.2 Data Wrangling Introduction](#1.2.2_wrangling)
    

* [1.3 Imports](#1.3_imports)


* [1.4 Load Data](#1.4_loaddata)


* [1.5 Fashion Dataset](#1.5_exploredata)

    * [1.5.1 Fashion data features](#1.5.1_feat)
    
    * [1.5.2 Fashion missing data](#1.5.2_f_missing)

            
        
* [1.6 Save Data](#1.6)
    
    * [1.6.1 Creating a parsing function](#1.6.1)
    
    * [1.6.2 Fashion CSV](#1.6.2)
        

* [1.7 Summary](#1.7_summary)

## 1.1 Data Wrangling 

## 1.2 Introduction<a name="1.2_introduction"></a>
In this notebook, we are going to primarily be focusing on loading the data, examining missing and duplicated values, and dropping unneccesary features. We may do a small amount of data cleaning, but we do not want to prematurely drop features that we may later need.

### 1.2.1 Problem Recap <a name="1.2.1_recap"></a>

Using customer text data about amazon products, we will build, evaluate and compare models to estimate the probability that a given text review can be classified as “positive”, “neutral”, or “negative”.

Our goal is to build a text classifier using Amazon product review data which can be used to analyze customer sentiment which does not have accompanying numeric data.

### 1.2.2 Data Wrangling Introduction <a name="1.2.2_wrangling"></a>
We will read in the data, remove unnecessary columns/features, and then save it in an easier-to-work-with tabular format (CSV). 

## 1.3 Imports <a name="1.3_imports"></a>

In [1]:
import json
import csv
import pandas as pd

## 1.4 Load Data<a name="1.4_loaddata"></a>
Datasets are sourced from:

[Jianmo Ni, Jiacheng Li, Julian McAuley
Empirical Methods in Natural Language Processing (EMNLP), 2019](https://nijianmo.github.io/amazon/index.html)

The data is split up by product category. 

### Fashion Category (883,636 reviews)

The data is available as compressed JSON files. We will use pandas to load the fashion data and determine which features to keep or drop and which features need preliminary cleaning.

We will explore the features and build a set of functions to parse the file. We will stream the JSON data, drop the values/features we do not want to keep, and save to separate CSV files.

## 1.5 Explore the Fashion data <a name="1.5_exploredata"></a>

Let's take a look at the top of the **Fashion** dataset

In [2]:
fashion_head = pd.read_json("../data/raw/AMAZON_FASHION.json", lines=True, nrows=200)

In [3]:
fashion_head.head(3)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN


In [4]:
fashion_head.dtypes

overall             int64
verified             bool
reviewTime         object
reviewerID         object
asin               object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote              float64
style              object
image              object
dtype: object

### 1.5.1 Fashion data features <a name="1.5.1_feat"></a>

The **overall** feature is the number of stars, which we'll be using for our categorical target feature (positive/neutral/negative). It's an integer from 1-5. 

The **verified** feature is a boolean and determines whether Amazon can confirm the user purchased the item they're reviewing (not a free/discounted reviewed item).

The **reviewTime** is in a date string format, and we'll drop it and keep the **unixReviewTime** instead.

The **reviewerID** and **reviewerName** have similar information and we can keep just one (the ID should be fine).

The **asin** is the specific product ID. We could keep it if we want to get item metadata later and use that for additional exploratory analysis (ie. price, product tags, product subcategory).

The **summary** is a user provided (or amazon generated) short item summary.

The **vote** column seems to indicate how many users found a review useful.

### Important features:

The **reviewText** will be the main input feature/features we will be using for our sentiment prediction and modelling.

Our target feature will be based on the **overall** column. We will be using the star value for each review to convert the data to Negative/Neutral/Positive categories. This will be the feature that our model trains and predicts with.

Let's check the **style** column.

In [5]:
fashion_head[~(fashion_head["style"].isna())]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
7,3,True,"09 22, 2013",A1BB77SEBQT8VX,B00007GDFV,Darrow H Ankrum II,mother - in - law wanted it as a present for h...,bought as a present,1379808000,NaN,{'Color:': ' Black'},NaN
8,3,True,"07 17, 2013",AHWOW7D1ABO9C,B00007GDFV,rosieO,"Item is of good quality. Looks great, too. But...",Buxton heiress collection,1374019200,NaN,{'Color:': ' Black'},NaN
9,3,True,"04 13, 2013",AKS3GULZE0HFC,B00007GDFV,M. Waltman,I had used my last el-cheapo fake leather ciga...,Top Clasp Broke Within 3 days!,1365811200,NaN,{'Color:': ' Black'},NaN
10,4,True,"03 9, 2013",A38NS6NF6WPXS,B00007GDFV,BTDoxies,This brand has been around a long time and you...,BUXTON QUALITY!,1362787200,NaN,{'Color:': ' Black'},NaN
11,2,True,"01 27, 2013",A1KOKO3HTSAI1H,B00007GDFV,Robin Howard,I smoke 100's and these are NOT made for them....,Buxton Heiress Collection Black Leather Cigare...,1359244800,NaN,{'Color:': ' Black'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
175,5,True,"08 28, 2016",A3U8IMV1FKI5Y3,B00008JPRZ,SPB,Love shirt and even makes me look good!,Love the shirt!!!,1472342400,NaN,"{'Size:': ' 16.5 - 34', 'Color:': ' White'}",NaN
176,5,True,"05 25, 2016",A21CTBULJC61N1,B00008JPRZ,Harrycarnival,great shirt,Five Stars,1464134400,NaN,"{'Size:': ' 17.5 - 37', 'Color:': ' White'}",NaN
193,5,False,"03 2, 2006",A3JGLFGCB7KZQX,B0000AWXMM,DG,"It was exactly as described. Beautiful ring, ...",BEAUTIFUL RING!!!! NO REGRETS WHATSOEVER,1141257600,NaN,"{'Size:': ' 12', 'Metal Type:': ' yellow-gold'}",NaN
194,4,False,"08 22, 2005",A179HMPWBPTJ2I,B0000AWXMM,Leana J. Kemp,This is a nice ring. It is simple and elegant...,White Gold Ring,1124668800,6.0,"{'Size:': ' 10', 'Metal Type:': ' white-gold'}",NaN


The **style** column seems to have sizing and color info but is missing for most reviews.

### 1.5.2 Fashion missing data <a name="1.5.2_f_missing"></a>

In [6]:
fashion = pd.read_json("../data/raw/AMAZON_FASHION.json", lines=True, chunksize=100000)

Using a json_reader iterator from Pandas, we can iterate through the data rather than loading the entire dataset into memory


In [7]:
missing_stats=[]
for chunk in fashion:
    missing_stats.append(chunk.isna().sum())

In [8]:
missing_vals = pd.DataFrame(missing_stats)

In [9]:
missing_vals

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,0,0,0,0,0,6,109,39,0,90595,28037,98423
1,0,0,0,0,0,18,108,61,0,89490,49765,97805
2,0,0,0,0,0,13,173,68,0,89152,44147,95963
3,0,0,0,0,0,14,172,77,0,86690,41255,94941
4,0,0,0,0,0,6,93,26,0,92429,78710,98412
5,0,0,0,0,0,5,123,49,0,93244,88410,97702
6,0,0,0,0,0,7,174,78,0,92668,87838,96291
7,0,0,0,0,0,11,152,76,0,92517,87214,95966
8,0,0,0,0,0,12,129,59,0,76951,73691,79326


In [10]:
missing_vals.sum()

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName          92
reviewText          1233
summary              533
unixReviewTime         0
vote              803736
style             579067
image             854829
dtype: int64

We have no missing values in our **overall** column, which is excellent. The **vote**, **style** and **image** columns have a lot of missing values, which is fine as we'll be dropping those columns. There are not many missing **reviewText** values.

Our final column/feature list to keep will be:

* **overall**
* **reviewerID**
* **verified**
* **asin**
* **reviewText**
* **summary**
* **unixReviewTime**

In [11]:
cols_to_keep = ["overall",  "verified", "reviewerID", "asin", "reviewText", "summary", "unixReviewTime"]

Let's do a quick preview to make sure our column list is correct.

In [12]:
fashion_head[cols_to_keep].head()

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A1D4G1SNUZWQOT,7106116521,Exactly what I needed.,perfect replacements!!,1413763200
1,2,True,A3DDWDH9PX2YX2,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400
2,4,False,A2MWC41EW7XL15,7106116521,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800
3,2,True,A2UH2QQ275NV45,7106116521,too tiny an opening,Two Stars,1408838400
4,3,False,A89F3LQADZBS5,7106116521,Okay,Three Stars,1406419200


## 1.8 Save Data <a name="1.8"></a>

### 1.8.1 Creating a parsing function <a name="1.8.1"></a>

In [13]:
def df_cleaner(temp_df_chunk: pd.DataFrame,
               columns_to_keep=cols_to_keep) -> pd.DataFrame:
    
    """This function will take in a chunk of our dataset as a dataframe.
       It will return only the columns we wish.
       
       Takes:
           temp_df_chunk: a dataframe to be edited
           columns_to_keep: the list of columns we wish to keep from that df          
       Returns:
           clean_df: a smaller df with drop_by_row NaNs dropped and only a subset of original columns
       """
  
    return temp_df_chunk[columns_to_keep]

Let's do a quick test.

In [18]:
fashion_test = pd.read_json("../data/raw/AMAZON_FASHION.json", lines=True, nrows=10)

df_cleaner(fashion_test.head())

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A1D4G1SNUZWQOT,7106116521,Exactly what I needed.,perfect replacements!!,1413763200
1,2,True,A3DDWDH9PX2YX2,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400
2,4,False,A2MWC41EW7XL15,7106116521,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800
3,2,True,A2UH2QQ275NV45,7106116521,too tiny an opening,Two Stars,1408838400
4,3,False,A89F3LQADZBS5,7106116521,Okay,Three Stars,1406419200


### 1.8.2 Fashion CSV <a name="1.8.2"></a>

In [35]:
fashion = pd.read_json("../data/raw/AMAZON_FASHION.json", lines=True)

In [33]:
i = 0
for chunk in fashion:
    path = f"../data/edited/fashion{i}.parquet"
    print(chunk.head())
    print(path)
    df_cleaner(chunk).to_parquet(path)
    i += 1

   overall  verified   reviewTime      reviewerID        asin  reviewerName  \
0        5      True  10 20, 2014  A1D4G1SNUZWQOT  7106116521         Tracy   
1        2      True  09 28, 2014  A3DDWDH9PX2YX2  7106116521     Sonja Lau   
2        4     False  08 25, 2014  A2MWC41EW7XL15  7106116521      Kathleen   
3        2      True  08 24, 2014  A2UH2QQ275NV45  7106116521   Jodi Stoner   
4        3     False  07 27, 2014   A89F3LQADZBS5  7106116521  Alexander D.   

                                          reviewText  \
0                             Exactly what I needed.   
1  I agree with the other review, the opening is ...   
2  Love these... I am going to order another pack...   
3                                too tiny an opening   
4                                               Okay   

                                             summary  unixReviewTime  vote  \
0                             perfect replacements!!      1413763200   NaN   
1  I agree with the other review

For space saving and quicker computations, we're using the parquet file format instead of csv.

In [35]:
!ls ../data/edited/

fashion0.parquet


In [22]:
pd.read_parquet("../data/edited/fashion0.parquet").head()

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A1D4G1SNUZWQOT,7106116521,Exactly what I needed.,perfect replacements!!,1413763200
1,2,True,A3DDWDH9PX2YX2,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400
2,4,False,A2MWC41EW7XL15,7106116521,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800
3,2,True,A2UH2QQ275NV45,7106116521,too tiny an opening,Two Stars,1408838400
4,3,False,A89F3LQADZBS5,7106116521,Okay,Three Stars,1406419200


In [23]:
!ls ../data/edited/

fashion0.parquet  fashion.parquet


## 1.9 Summary <a name="1.9_summary"></a>

We have done initial cleaning of the dataset. We did not drop any rows from our data yet, as we only had a small number of missing **reviewText** data, which we will explore in the future.

We dropped the **reviewTime** and **reviewerName** columns as they're somewhat duplicated by our **unixReviewTime** and **reviewerID** variables. We dropped the **vote**, **style**, and **image** columns, as they're both largely filled with missing values and unimportant to our analysis generally.

Feature variable: **reviewText** will be our main data source for analysis, though the review summary also contains text, possibly from the user, about their feelings about the item.

Target variable: **overall** stars (1-5) will be converted in the next part of the project into categorical variables, to predict positive, neutral and negative feelings regarding user's opinions of items.